In [30]:
!pip install pandas
!pip install scikit-learn

In [2]:
import pandas as pd

In [23]:
df = pd.read_csv('..\data\Missing_Migrants_Global_Figures_allData.csv')
ignore_cols = [
    "Main ID", 
    "Incident ID", 
    "Incident Type", 
    "Incident Year", 
    "Month", 
    "Coordinates", 
    "UNSD Geographical Grouping", 
    "Information Source"
]
df = df.drop(columns=ignore_cols)
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rasan\AppData\Local\Temp\ipykernel_23600\1018048934.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('..\data\Missing_Migrants_Global_Figures_allData.csv')


,Region of Incident,Incident Date,Number of Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Country of Origin,Region of Origin,Cause of Death,Country of Incident,Migration Route,Location of Incident,URL,Source Quality
0,North America,2014-01-06,1.0,NaN,1,NaN,NaN,1.0,NaN,Guatemala,Central America,Mixed or unknown,United States of America,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,http://humaneborders.info/,5
1,North America,2014-01-12,1.0,NaN,1,NaN,NaN,NaN,NaN,Unknown,Latin America / Caribbean (P),Mixed or unknown,United States of America,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,NaN,5
2,North America,2014-01-14,1.0,NaN,1,NaN,NaN,NaN,NaN,Unknown,Latin America / Caribbean (P),Mixed or unknown,United States of America,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,NaN,5
3,North America,2014-01-16,1.0,NaN,1,NaN,NaN,1.0,NaN,Mexico,Central America,Violence,United States of America,US-Mexico border crossing,"near Douglas, Arizona, USA",http://bit.ly/1qfIw00,5
4,Europe,2014-01-16,1.0,0.0,1,2.0,NaN,1.0,NaN,Sudan,Northern Africa,Harsh environmental conditions / lack of adequ...,Russian Federation,NaN,Border between Russia and Estonia,http://bit.ly/1rTFTjR,1


In [ ]:
import sys
import os
sys.path.append('../../news_scraping/src')
from scraper import Scraper

path = '../data/articles_dataset.csv'
file_exists = os.path.isfile(path)

article_scraper = Scraper()
for index, row in df.iterrows():
    print(f"Reading Article {index+1}")
    article = article_scraper.get_article_by_url(row["URL"])
    if article:
        df.at[index, 'Content'] = article.text  # Append content if scraping was successful
        row_data = df.loc[[index]].copy()
        row_data.to_csv(path, mode='a', header=False, index=False)
        print(f"Appended article {index+1}")
    else:
        df.drop(index, inplace=True)
df.head()

In [24]:
df_cols = df.columns.to_list()
df_cols.append("Content")

df = pd.read_csv('../data/articles_dataset.csv', header=None)
df.columns = df_cols
df.to_csv('../data/articles_dataset2.csv')
df.head()

,Region of Incident,Incident Date,Number of Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Country of Origin,Region of Origin,Cause of Death,Country of Incident,Migration Route,Location of Incident,URL,Source Quality,Content
0,North America,2014-01-06,1.0,NaN,1,NaN,NaN,1.0,NaN,Guatemala,Central America,Mixed or unknown,United States of America,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,http://humaneborders.info/,5,This web site is the result of ongoing partner...
1,North America,2014-01-16,1.0,NaN,1,NaN,NaN,1.0,NaN,Mexico,Central America,Violence,United States of America,US-Mexico border crossing,"near Douglas, Arizona, USA",http://bit.ly/1qfIw00,5,NaN
2,Mediterranean,2014-01-19,12.0,NaN,12,NaN,9.0,NaN,3.0,"Afghanistan,Syrian Arab Republic",Unknown,Drowning,Greece,Eastern Mediterranean,Waters near Greece while being towed back to T...,http://bit.ly/2aMCwfg,5,Nine child­ren and three women died when their...
3,Europe,2014-01-30,1.0,0.0,1,2.0,NaN,1.0,1.0,Iran (Islamic Republic of),Southern Asia,Vehicle accident / death linked to hazardous t...,France,English Channel to the UK,France - Calais,http://bit.ly/1icTIF9,4,On the night of 30th January 2014 a 17 year ol...
4,Caribbean,2014-01-30,1.0,NaN,1,NaN,NaN,1.0,1.0,Haiti,Caribbean,Drowning,United States of America,Caribbean to US,"Off the coast of Fort Lauderdale, Florida",http://bit.ly/1zU2LSq,1,"FORT LAUDERDALE, Fla. – Authorities have calle..."


In [29]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, -1:]
y = df.iloc[:, :-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting splits
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

2456